# Improved LSTM baseline

This kernel is a somewhat improved version of [Keras - Bidirectional LSTM baseline](https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051) along with some additional documentation of the steps. (NB: this notebook has been re-run on the new test set.)

In [1]:
# Fast Text
# Increase the glove Embedding
# Use Fast Text to generate the embedding

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

import fastText

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

import matplotlib.pyplot as plt
%matplotlib inline  

Using TensorFlow backend.


In [4]:
# 2 Options
# Sentence Vector
# word Vector generated from Fast Text n-grams
# https://github.com/keras-team/keras/issues/1638

# def generate_data(directory, batch_size):
#     """Replaces Keras' native ImageDataGenerator."""
#     i = 0
#     file_list = os.listdir(directory)
#     while True:
#         image_batch = []
#         for b in range(batch_size):
#             if i == len(file_list):
#                 i = 0
#                 random.shuffle(file_list)
#             sample = file_list[i]
#             i += 1
#             image = cv2.resize(cv2.imread(sample[0]), INPUT_SHAPE)
#             image_batch.append((image.astype(float) - 128) / 128)

#         yield np.array(image_batch)

# def generator(X_data, y_data, batch_size):

#   samples_per_epoch = X_data.shape[0]
#   number_of_batches = samples_per_epoch/batch_size
#   counter=0

#   while 1:

#     X_batch = np.array(X_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
#     y_batch = np.array(y_data[batch_size*counter:batch_size*(counter+1)]).astype('float32')
#     counter += 1
#     yield X_batch,y_batch

#     #restart counter to yeild data in the next epoch as well
#     if counter <= number_of_batches:
#         counter = 0
        
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def next(self):
        with self.lock:
            return self.it.next()


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def myGenerator(X,y):
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    y_train = np_utils.to_categorical(y_train,10)
    X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
    X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    while 1:
        for i in range(1875):
            print("came till here")
            yield X_train[i*32:(i+1)*32], y_train[i*32:(i+1)*32]
            print("and here")
        print("and here too")

model.fit_generator(my_generator, samples_per_epoch = 5000, nb_epoch = 2, verbose=2, show_accuracy=True, callbacks=[pb], validation_data=None, class_weight=None, nb_worker=2)        
model.fit_generator(generator(dataFrameTrain,expectedFrameTrain,batch_size), epochs=3,steps_per_epoch = dataFrame.shape[0]/batch_size, validation_data=generator(dataFrameTest,expectedFrameTest,batch_size*2),validation_steps=dataFrame.shape[0]/batch_size*2)

In [10]:
list_sentences_train[0]

"explanation why the edits made under my username hardcore metallica fan were reverted ?  they weren ' t vandalisms ,  just closure on some gas after i voted at new york dolls fac .  and please don ' t remove the template from the talk page since i ' m retired now .  _ip_ "

In [2]:
ft = fastText.load_model('wv/wiki.en.bin')

In [5]:
def normalize(s):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    s = s.lower()
    # Replace ips
    s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([\'\"\.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    
    return s

In [4]:
ft.get_word_vector('test').shape

(300,)

We include the GloVe word vectors in our input files. To include these in your kernel, simple click 'input files' at the top of the notebook, and search 'glove' in the 'datasets' section.

In [6]:
path = 'data/'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

Set some basic config parameters:

In [7]:
embed_size = 300 # how big is each word vector
max_features = 500000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 500 # max number of words in a comment to use

Read in our data and replace missing values:

In [8]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)
train["comment_text"].fillna("_empty_",inplace=True)
test["comment_text"].fillna("_empty_",inplace=True)

list_sentences_train = train["comment_text"].apply(lambda x:normalize(x)).values

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].apply(lambda x:normalize(x)).values

Standard keras preprocessing, to turn each comment into a list of word indexes of equal length (with truncation or padding as needed).

In [8]:
# tokenizer = Tokenizer(num_words=max_features)
tokenizer = Tokenizer(oov_token='_oov_')
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

Read the glove word vectors (space delimited strings) into a dictionary from word->vector.

Use these vectors to create our embedding matrix, with random initialization for words that aren't in GloVe. We'll use the same mean and stdev of embeddings the GloVe has when generating the random init.

In [9]:
word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))+1
nb_words = len(word_index)+1
embedding_matrix = np.random.normal(-0.0039050116, 0.38177028, (nb_words, embed_size))
for word, i in word_index.items():
    embedding_matrix[i] = ft.get_word_vector(word).astype('float32') # out of word vocabulary

In [10]:
embedding_matrix.shape

(192857, 300)

Simple bidirectional LSTM with two fully connected layers. We add some dropout to the LSTM since even 2 epochs is enough to overfit.

In [11]:
inp = Input(shape=(maxlen,))
x = Embedding(nb_words, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we're ready to fit out model! Use `validation_split` when not submitting.

In [12]:
# model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);
model.fit(X_t, y, batch_size=32, epochs=2);

Epoch 1/2
159571/159571 [==============================] - 4826s 30ms/step - loss: 0.0516 - acc: 0.9811
Epoch 2/2
159571/159571 [==============================] - 4838s 30ms/step - loss: 0.0355 - acc: 0.9856


And finally, get predictions for the test set and prepare a submission CSV:

In [13]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('fast_text_baseline_3_norm.csv', index=False)

153164/153164 [==============================] - 44s 290us/step


In [4]:
# sample_submission.to_csv('base_test.csv',index=False)

In [19]:
# test_submission = pd.read_csv('data/sample_submission.csv')
# len(test_submission)

In [ ]:
# Baseline Score
# loss: 0.0417 - acc: 0.9840 - val_loss: 0.0451 - val_acc: 0.9829 --> AUC : 0.9787

